In [4]:
#!pip install bitsandbytes

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_id = "Trendyol/Trendyol-LLM-7B-chat-v4.1.0"


# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  # ya da "fp4"
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Model yükleniyor
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="cuda",
    use_cache=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
"""
# Sistem ve kullanıcı mesajlarını birleştir
DEFAULT_SYSTEM_PROMPT = "Sen yardımsever bir asistansın ve sana verilen talimatlar doğrultusunda en iyi cevabı üretmeye çalışacaksın."
user_message = "Naıslsın?"

# Prompta uygun formatta string oluştur
full_prompt = f"<|system|>\n{DEFAULT_SYSTEM_PROMPT}\n<|user|>\n{user_message}\n<|assistant|>\n"

# Tokenize et
inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

# Sampling ayarları
generation_kwargs = dict(
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.3,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.1
)

# Metin üretimi
with torch.no_grad():
    outputs = model.generate(**inputs, **generation_kwargs)

# Tokenları string'e çevir
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Yalnızca cevabı al
answer = generated_text.replace(full_prompt, "").strip()
print(answer)
"""

config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

'\n# Sistem ve kullanıcı mesajlarını birleştir\nDEFAULT_SYSTEM_PROMPT = "Sen yardımsever bir asistansın ve sana verilen talimatlar doğrultusunda en iyi cevabı üretmeye çalışacaksın."\nuser_message = "Naıslsın?"\n\n# Prompta uygun formatta string oluştur\nfull_prompt = f"<|system|>\n{DEFAULT_SYSTEM_PROMPT}\n<|user|>\n{user_message}\n<|assistant|>\n"\n\n# Tokenize et\ninputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)\n\n# Sampling ayarları\ngeneration_kwargs = dict(\n    max_new_tokens=1024,\n    do_sample=True,\n    temperature=0.3,\n    top_k=50,\n    top_p=0.9,\n    repetition_penalty=1.1\n)\n\n# Metin üretimi\nwith torch.no_grad():\n    outputs = model.generate(**inputs, **generation_kwargs)\n\n# Tokenları string\'e çevir\ngenerated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)\n\n# Yalnızca cevabı al\nanswer = generated_text.replace(full_prompt, "").strip()\nprint(answer)\n'

In [9]:
tools = [
    {
        "name": "get_city_count",
        "description": "Get current city count of given country.",
        "parameters": {
            "type": "object",
            "properties": {
                "country_name": {
                    "type": "string",
                    "description": "The name of the country to get the count for.",
                },
            },
            "required": ["country_name"],
        },
    },
    {
        "name": "get_temperature_date",
        "description": "Get temperature at a location and date.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": 'The location to get the temperature for, in the format "City, State, Country".',
                },
                "date": {
                    "type": "string",
                    "description": 'The date to get the temperature for, in the format "Year-Month-Day".',
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": 'The unit to return the temperature in. Defaults to "celsius".',
                },
            },
            "required": ["location", "date"],
        },
    },
    {
        "name": "get_currency_exchange",
        "description": "Get current exchange rate between two currencies.",
        "parameters": {
            "type": "object",
            "properties": {
                "base_currency": {
                    "type": "string",
                    "description": "The currency you want to convert from (e.g., USD).",
                },
                "target_currency": {
                    "type": "string",
                    "description": "The currency you want to convert to (e.g., EUR).",
                },
            },
            "required": ["base_currency", "target_currency"],
        },
    },
    {
        "name": "get_country_info",
        "description": "Retrieve information about a specific country.",
        "parameters": {
            "type": "object",
            "properties": {
                "country_name": {
                    "type": "string",
                    "description": "The full name of the country (e.g., Turkey, Germany).",
                },
            },
            "required": ["country_name"],
        },
    },
    {
        "name": "get_weather_forecast",
        "description": "Get a weather forecast for a city for the next 5 days.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city name (and optionally state and country) to get the forecast for.",
                }
            },
            "required": ["location"],
        },
    },
    {
        "name": "get_news_headlines",
        "description": "Get the latest news headlines on a given topic.",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The subject or keyword to retrieve headlines for.",
                },
                "language": {
                    "type": "string",
                    "description": "Language of the news (e.g., en, tr). Default is en.",
                },
            },
            "required": ["topic"],
        },
    },
]

user_message = "ALMANYA nasıl bir yer?"
messages = [
    {"role": "system", "content": "Sen, aşağıdaki fonksiyonlara erişimi olan yardımcı bir asistansın. Gerektiğinde bunları kullanabilirsin -"},
    {"role": "user", "content": user_message}
]

text = tokenizer.apply_chat_template(messages, tools=tools, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)
output_text = tokenizer.batch_decode(outputs)[0][len(text):]
print(output_text)
# '<function>{"name": "get_city_count", "arguments": \'{"country_name": "Turkey"}\'}</function><|im_end|>'


<function>{"name": "get_country_info", "arguments": '{"country_name": "Germany"}'}</function><|im_end|>


In [10]:
import json
import re

# 🔧 Model çıktısı (senin örneğin)

raw_output = output_text
# 1. <function>...</function> içeriğini çek
match = re.search(r"<function>(.*?)</function>", raw_output)
if not match:
    raise ValueError("Fonksiyon çağrısı bulunamadı!")

# 2. Dıştaki JSON'u parse et (önce çift tırnak düzeltmeleri)
function_str = match.group(1)
function_str_clean = function_str.replace("'", '"')  # tek tırnakları çift tırnağa çevir
function_str_clean = re.sub(r'(?<!\\)"({.*?})"(?!\\)', lambda m: m.group(1).replace('"', '\\"'), function_str_clean)

# 3. Dıştaki JSON’u yükle
try:
    function_call = json.loads(function_str)
except json.JSONDecodeError:
    # Bazı model çıktıları çift encode olabilir; elle çöz
    name_match = re.search(r'"name"\s*:\s*"([^"]+)"', function_str)
    args_match = re.search(r'"arguments"\s*:\s*\'({.*})\'', function_str)

    function_name = name_match.group(1)
    arguments_raw = args_match.group(1)
    arguments = json.loads(arguments_raw)
else:
    function_name = function_call["name"]
    arguments = json.loads(function_call["arguments"])

def get_city_count(country_name):
    data = {
        "turkey": 81,
        "germany": 16,
        "france": 18,
        "usa": 50
    }
    return {"city_count": data.get(country_name.lower(), "bilinmiyor")}


def get_temperature_date(location, date, unit="celsius"):
    # Örnek: sabit değer döndürüyoruz
    return {"temperature": 22, "unit": unit}


def get_currency_exchange(base_currency, target_currency):
    exchange_rates = {
        ("usd", "eur"): 0.93,
        ("eur", "usd"): 1.08,
        ("usd", "try"): 32.4,
        ("try", "usd"): 0.031
    }
    key = (base_currency.lower(), target_currency.lower())
    rate = exchange_rates.get(key, "rate not available")
    return {"exchange_rate": rate}


def get_country_info(country_name):
    countries = {
        "turkey": {
            "capital": "Ankara",
            "currency": "TRY",
            "population": "85M",
            "continent": "Asia/Europe"
        },
        "germany": {
            "capital": "Berlin",
            "currency": "EUR",
            "population": "83M",
            "continent": "Europe"
        }
    }
    return countries.get(country_name.lower(), {"error": "Ülke bilgisi bulunamadı"})


def get_weather_forecast(location):
    # Örnek 5 günlük hava durumu
    return {
        "location": location,
        "forecast": [
            {"day": "Monday", "temperature": 22},
            {"day": "Tuesday", "temperature": 24},
            {"day": "Wednesday", "temperature": 23},
            {"day": "Thursday", "temperature": 20},
            {"day": "Friday", "temperature": 21},
        ]
    }


def get_news_headlines(topic, language="tr"):
    # Sahte haber başlıkları örneği
    headlines = {
        "ai": [

        "AI dünyayı dönüştürüyor",
        "AI araştırmalarında yeni atılımlar",
        "OpenAI yeni GPT modelini yayınladı"
        ],
        "economics": [

        "Küresel piyasalarda oynaklık görülüyor",
        "Enflasyon oranları istikrar kazanıyor",
        "IMF büyüme projeksiyonlarını yayınladı"
        ]
    }
    return {
        "topic": topic,
        "language": language,
        "headlines": headlines.get(topic.lower(), ["No headlines found."])
    }

# 5. Fonksiyonu çalıştır

function_registry = {
    "get_city_count": get_city_count,
    "get_temperature_date": get_temperature_date,
    "get_currency_exchange": get_currency_exchange,
    "get_country_info": get_country_info,
    "get_weather_forecast": get_weather_forecast,
    "get_news_headlines": get_news_headlines,
}

try:
    function_result = function_registry.get(function_name, lambda **kwargs: {"error": "Bilinmeyen fonksiyon"})(**arguments)
except Exception as e:
    function_result = {"error": f"Fonksiyon çalıştırılırken hata oluştu: {str(e)}"}

# 6. Sonuç
print(function_result)


{'capital': 'Berlin', 'currency': 'EUR', 'population': '83M', 'continent': 'Europe'}


In [13]:
def call_model_with_tool_result(user_message, tool_name, tool_args, tool_output):
    full_prompt = (
        f"<|system|>\nSen yardımcı bir asistansın ve bazı fonksiyonlardan veri alabiliyorsun.\n"
        f"<|user|>\n{user_message}\n"
        f"<|assistant|>\n<function>{{\"name\": \"{tool_name}\", \"arguments\": \"{json.dumps(tool_args)}\"}}</function>\n"
        f"<|function|># {tool_name}\n{json.dumps(tool_output)}\n"
        f"<|assistant|>\n"
    )

    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)[len(full_prompt):]

response = call_model_with_tool_result(
    user_message=user_message,
    tool_name=function_name,
    tool_args=arguments,
    tool_output=json.dumps(function_result)
)
print(response)


Almanya, Avrupa'da yer alan bir ülkedir. Başkenti Berlin'dir ve resmi para birimi Euro'dur. Nüfusu yaklaşık 83 milyondur.
